# E7 - Project
Team member -
Rutuparn
Tanish
Vihang
Samarpit

# Reading images from the folder


In [1]:
import cv2
import os
import numpy as np

def load_and_process_images(folder_path='images', target_size=(800, 600)):
    """
    Loads all images from a folder.
    - Crops to 4:3 aspect ratio if needed.
    - Scales down to 800x600 if larger, but never scales up.
    - Draws 8x8 grid lines.
    - Saves to 'resized_images'.
    - Divides into 8x8 tiles (for further processing).
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    images = []
    resized_images = []
    divided_images = []

    # Create output folder
    output_folder = 'resized_images'
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder '{folder_path}' does not exist.")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            file_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"⚠️ Skipped unreadable image: {filename}")
                    continue

                h, w = img.shape[:2]
                aspect_ratio = w / h

                # --- a. Enforce 4:3 aspect ratio by cropping ---
                target_ratio = 4 / 3
                if abs(aspect_ratio - target_ratio) > 0.01:  # allow slight tolerance
                    if aspect_ratio > target_ratio:
                        # Image is too wide → crop width
                        new_w = int(h * target_ratio)
                        x_start = (w - new_w) // 2
                        img = img[:, x_start:x_start + new_w]
                    else:
                        # Image is too tall → crop height
                        new_h = int(w / target_ratio)
                        y_start = (h - new_h) // 2
                        img = img[y_start:y_start + new_h, :]

                # --- b. Scale down if larger than 800x600 ---
                h, w = img.shape[:2]
                if w > target_size[0] or h > target_size[1]:
                    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

                # --- c. Do NOT scale up smaller images ---
                final_img = img
                h, w = final_img.shape[:2]

                # Draw 8x8 grid lines
                tile_h, tile_w = h // 8, w // 8
                for i in range(1, 8):
                    # Horizontal lines
                    y = i * tile_h
                    cv2.line(final_img, (0, y), (w, y), (0, 255, 0), 1)
                    # Vertical lines
                    x = i * tile_w
                    cv2.line(final_img, (x, 0), (x, h), (0, 255, 0), 1)

                # Save processed image
                save_path = os.path.join(output_folder, filename)
                cv2.imwrite(save_path, final_img)

                # Keep copies in memory
                images.append(final_img.copy())
                resized_images.append(final_img)

                # Divide into 8x8 tiles (using actual dimensions)
                tiles = [
                    final_img[i * tile_h:(i + 1) * tile_h, j * tile_w:(j + 1) * tile_w]
                    for i in range(8)
                    for j in range(8)
                ]
                divided_images.append(tiles)

            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")

    return images, resized_images, divided_images


# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    images, resized_images, divided_images = load_and_process_images('images')

    print(f"✅ Loaded and processed {len(images)} images.")
    print(f"📁 Grid-marked images saved in the 'resized_images' folder.")
    if divided_images:
        print(f"Each image divided into {len(divided_images[0])} tiles (8x8).")


✅ Loaded and processed 465 images.
📁 Grid-marked images saved in the 'resized_images' folder.
Each image divided into 64 tiles (8x8).


# Conversion into 800x600 pixels


# dividing into 8x8 grid

Converting the images to greycode to reduce dimentionality

# Image processing to identify animals using HOG.

In [3]:
import cv2
import os
import numpy as np

def safe_imread(file_path):
    """
    Safely read image using OpenCV with fallback modes.
    """
    try:
        img = cv2.imread(file_path)
        if img is None:
            return cv2.imread(file_path, cv2.IMREAD_REDUCED_COLOR_2)
        return img
    except Exception:
        return cv2.imread(file_path, cv2.IMREAD_REDUCED_COLOR_4)


def crop_to_aspect_ratio(img, target_aspect=(4, 3)):
    """
    Crop the image to a target aspect ratio (4:3 by default).
    """
    h, w = img.shape[:2]
    target_ratio = target_aspect[0] / target_aspect[1]
    current_ratio = w / h

    if abs(current_ratio - target_ratio) < 1e-2:
        return img  # Already 4:3

    if current_ratio > target_ratio:
        # Too wide — crop horizontally
        new_w = int(h * target_ratio)
        start_x = (w - new_w) // 2
        cropped = img[:, start_x:start_x + new_w]
    else:
        # Too tall — crop vertically
        new_h = int(w / target_ratio)
        start_y = (h - new_h) // 2
        cropped = img[start_y:start_y + new_h, :]

    return cropped


def resize_image(img, max_size=(800, 600)):
    """
    Resize the image to a maximum of 800x600.
    Do not scale up smaller images.
    """
    h, w = img.shape[:2]
    max_w, max_h = max_size
    if w > max_w or h > max_h:
        return cv2.resize(img, (max_w, max_h), interpolation=cv2.INTER_AREA)
    return img


def load_and_save_grayscale_images(input_folder='images', output_folder='grey_images'):
    """
    Load all images from input_folder, crop to 4:3, resize down if needed,
    convert to grayscale, and save into output_folder.
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(supported_extensions):
            continue

        file_path = os.path.join(input_folder, filename)
        img = safe_imread(file_path)
        if img is None:
            print(f"⚠️ Skipped unreadable image: {filename}")
            continue

        # Step 1: Crop to 4:3
        img = crop_to_aspect_ratio(img, target_aspect=(4, 3))

        # Step 2: Resize only if larger than 800x600
        img = resize_image(img, max_size=(800, 600))

        # Step 3: Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Step 4: Save grayscale image
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, gray)

        print(f"✅ Saved grayscale image: {filename}")

    print(f"\n🎯 All grayscale images saved in '{output_folder}/'.")


# Example usage
if __name__ == "__main__":
    load_and_save_grayscale_images('images', 'grey_images')


✅ Saved grayscale image: IMG_3371.JPG
✅ Saved grayscale image: DSC00322.JPG
✅ Saved grayscale image: CIMG0364.JPG
✅ Saved grayscale image: DSC00096.JPG
✅ Saved grayscale image: CIMG0014.JPG
✅ Saved grayscale image: CIMG0367.JPG
✅ Saved grayscale image: IMG_3637.JPG
✅ Saved grayscale image: IMG_4164.JPG
✅ Saved grayscale image: CIMG0980.jpg
✅ Saved grayscale image: IMG_3569(1).JPG
✅ Saved grayscale image: IMG_3334(1).JPG
✅ Saved grayscale image: CIMG0100.JPG
✅ Saved grayscale image: CIMG0113~2.JPG
✅ Saved grayscale image: IMG_20180724_170028451.jpg
✅ Saved grayscale image: CIMG0140(1).JPG
✅ Saved grayscale image: CIMG0125.JPG
✅ Saved grayscale image: CIMG0080(1).JPG
✅ Saved grayscale image: DSC00343.JPG
✅ Saved grayscale image: CIMG0162.JPG
✅ Saved grayscale image: CIMG0493.JPG
✅ Saved grayscale image: IMG_3858.JPG
✅ Saved grayscale image: IMG_20180726_154917733_BURST000_COVER_TOP.jpg
✅ Saved grayscale image: 26102010062.jpg
✅ Saved grayscale image: CIMG0569(1).JPG
✅ Saved grayscale ima

In [1]:
import cv2
import os
import numpy as np

def draw_grid_on_gray_image(img, grid_size=(8, 8)):
    """
    Draw an 8x8 grid with small numbers in the bottom-left corner of each cell.
    Works for grayscale images.
    """
    h, w = img.shape[:2]
    grid_h, grid_w = grid_size
    dy, dx = h // grid_h, w // grid_w

    # Convert grayscale to BGR to draw colored text/lines
    img_colored = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    cell_num = 1
    for i in range(grid_h):
        for j in range(grid_w):
            x, y = j * dx, i * dy
            # Draw green grid rectangles
            cv2.rectangle(img_colored, (x, y), (x + dx, y + dy), (0, 255, 0), 1)
            # Draw small red number on bottom-left of each cell
            cv2.putText(img_colored, str(cell_num),
                        (x + 4, y + dy - 4),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1, cv2.LINE_AA)
            cell_num += 1

    return img_colored


def process_grey_images_with_grid(input_folder='grey_images', output_folder='grey_images_with_grid'):
    """
    Loads grayscale images, draws 8x8 grid with numbering, and saves them to output_folder.
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(supported_extensions):
            continue

        file_path = os.path.join(input_folder, filename)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"⚠️ Skipped unreadable image: {filename}")
            continue

        # Draw grid and numbering
        grid_img = draw_grid_on_gray_image(img, grid_size=(8, 8))

        # Save to output folder
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, grid_img)
        print(f"✅ Saved grid image: {filename}")

    print(f"\n🎯 All images with grid saved to '{output_folder}/'.")


# Example usage
if __name__ == "__main__":
    process_grey_images_with_grid('grey_images', 'grey_images_with_grid')


✅ Saved grid image: 100820081692.jpg
✅ Saved grid image: 100820081693.jpg
✅ Saved grid image: 100820081694.jpg
✅ Saved grid image: 100820081695.jpg
✅ Saved grid image: 141120142185.jpg
✅ Saved grid image: 26102010062.jpg
✅ Saved grid image: 29032008693.jpg
✅ Saved grid image: 29032008695.jpg
✅ Saved grid image: 29032008696.jpg
✅ Saved grid image: 29032008701.jpg
✅ Saved grid image: CIMG0001~2.JPG
✅ Saved grid image: CIMG0002.JPG
✅ Saved grid image: CIMG0005.JPG
✅ Saved grid image: CIMG0006.JPG
✅ Saved grid image: CIMG0011.JPG
✅ Saved grid image: CIMG0012~2.JPG
✅ Saved grid image: CIMG0014(1).JPG
✅ Saved grid image: CIMG0014.JPG
✅ Saved grid image: CIMG0015.JPG
✅ Saved grid image: CIMG0017.JPG
✅ Saved grid image: CIMG0018.JPG
✅ Saved grid image: CIMG0021~2.JPG
✅ Saved grid image: CIMG0029.JPG
✅ Saved grid image: CIMG0030.JPG
✅ Saved grid image: CIMG0034.JPG
✅ Saved grid image: CIMG0035.JPG
✅ Saved grid image: CIMG0047.JPG
✅ Saved grid image: CIMG0054.JPG
✅ Saved grid image: CIMG0056.JPG

In [15]:
import os
import csv
import glob
import math
import json
import joblib
import numpy as np
from pathlib import Path
from collections import defaultdict

import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score

# -------------------------
# CONFIG
# -------------------------
DATA_DIR = Path(".")
IMG_DIR = DATA_DIR / "resized_images"          # training images
LABEL_CSV = DATA_DIR / "grid_overlaps.csv"     # labels
MODEL_DIR = DATA_DIR / "models"
MODEL_DIR.mkdir(exist_ok=True)

# Grid + image geometry
IMG_W, IMG_H = 800, 600
GRID_W, GRID_H = 8, 8
CELL_W, CELL_H = IMG_W // GRID_W, IMG_H // GRID_H  # 100 x 75

# Label indexing convention
POS_INDICES_ARE_ONE_BASED = False  # <- set True if your CSV uses 1..64

# HOG params (good starting point for 100x75 cells)
HOG_PIXELS_PER_CELL = (16, 16)
HOG_CELLS_PER_BLOCK = (2, 2)
HOG_ORIENTATIONS    = 9
HOG_BLOCK_NORM      = "L2-Hys"

# Color histogram params (HSV)
USE_COLOR_HIST = True
H_BINS, S_BINS, V_BINS = 16, 8, 8
PAD_X, PAD_Y = 20, 15 

RANDOM_SEED = 13
N_JOBS = -1  # not used by LinearSVC; kept for future changes


In [18]:
import csv
from pathlib import Path

def read_labels(label_csv: Path):
    """
    Reads rows like:
      image_name,grid_indices
      100820081693.jpg,"18 19 20 21 ..."
    Returns: {filename -> set of positive cell indices (0..63)}.
    """
    labels = {}
    with open(label_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        # Accept either column name just in case
        name_key  = "image_name" if "image_name" in reader.fieldnames else "filename"
        idx_key   = "grid_indices" if "grid_indices" in reader.fieldnames else reader.fieldnames[1]

        for rownum, row in enumerate(reader, start=2):  # header = line 1
            fname = (row.get(name_key) or "").strip()
            idx_str = (row.get(idx_key) or "").strip()

            if not fname:
                print(f"[warn] line {rownum}: empty image_name, skipping")
                continue

            # Accept either space or comma separated numbers (your file is space-separated)
            tokens = idx_str.replace(",", " ").split()
            ints = []
            for t in tokens:
                try:
                    ints.append(int(t))
                except ValueError:
                    print(f"[warn] line {rownum}: non-integer token '{t}' ignored")

            if POS_INDICES_ARE_ONE_BASED:
                ints = [i-1 for i in ints]

            # keep only 0..63
            clean = [i for i in ints if 0 <= i < GRID_W*GRID_H]
            if len(clean) != len(ints):
                bad = set(ints) - set(clean)
                print(f"[warn] line {rownum}: out-of-range indices {sorted(bad)} ignored for {fname}")

            labels[fname] = set(clean)

    if not labels:
        print("[warn] No labels parsed. Check CSV format.")

    return labels

def cell_index(r, c, grid_h=GRID_H, grid_w=GRID_W):
    """Row-major index 0..63."""
    return r * grid_w + c

def crop_cell(img, r, c):
    y0, y1 = r*CELL_H, (r+1)*CELL_H
    x0, x1 = c*CELL_W, (c+1)*CELL_W
    return img[y0:y1, x0:x1]

def hog_features(gray_patch):
    # skimage expects float [0,1] or uint8; we pass uint8
    feat = hog(
        gray_patch,
        orientations=HOG_ORIENTATIONS,
        pixels_per_cell=HOG_PIXELS_PER_CELL,
        cells_per_block=HOG_CELLS_PER_BLOCK,
        block_norm=HOG_BLOCK_NORM,
        feature_vector=True
    )
    return feat

def color_hist_features(bgr_patch):
    hsv = cv2.cvtColor(bgr_patch, cv2.COLOR_BGR2HSV)
    h_hist = cv2.calcHist([hsv],[0],None,[H_BINS],[0,180]).flatten()
    s_hist = cv2.calcHist([hsv],[1],None,[S_BINS],[0,256]).flatten()
    v_hist = cv2.calcHist([hsv],[2],None,[V_BINS],[0,256]).flatten()
    hist = np.concatenate([h_hist, s_hist, v_hist]).astype(np.float32)
    # normalize to unit sum (avoid scale issues)
    hist_sum = hist.sum() + 1e-8
    return (hist / hist_sum)
def extract_cell_features(img_bgr):
    """
    Extract features for all 64 cells using a fixed-size context pad around each cell.
    Every crop is exactly (CELL_H + 2*PAD_Y) x (CELL_W + 2*PAD_X).
    """
    # 1) Resize base image to canonical size
    img_resized = cv2.resize(img_bgr, (IMG_W, IMG_H))

    # 2) Pad the WHOLE image once (reflect padding avoids edge artifacts)
    img_pad = cv2.copyMakeBorder(
        img_resized,
        PAD_Y, PAD_Y, PAD_X, PAD_X,
        borderType=cv2.BORDER_REFLECT_101
    )
    gray_pad = cv2.cvtColor(img_pad, cv2.COLOR_BGR2GRAY)

    # Fixed crop dims for every cell
    crop_h = CELL_H + 2*PAD_Y
    crop_w = CELL_W + 2*PAD_X

    feats = []
    for r in range(GRID_H):
        for c in range(GRID_W):
            # Coordinates IN THE PADDED IMAGE:
            # start at original cell corner (no +PAD in the start),
            # end at (cell end + 2*PAD), so height/width are constant.
            y0 = r * CELL_H
            x0 = c * CELL_W
            y1 = (r + 1) * CELL_H + 2 * PAD_Y
            x1 = (c + 1) * CELL_W + 2 * PAD_X

            cell_bgr  = img_pad[y0:y1, x0:x1]
            cell_gray = gray_pad[y0:y1, x0:x1]

            # Safety assert (can remove after first run)
            # assert cell_bgr.shape[0] == crop_h and cell_bgr.shape[1] == crop_w, f"Crop mismatch at r={r}, c={c}: {cell_bgr.shape}"

            f_hog = hog_features(cell_gray)
            if USE_COLOR_HIST:
                f = np.concatenate([f_hog, color_hist_features(cell_bgr)])
            else:
                f = f_hog
            feats.append(f)

    return np.vstack(feats)  # (64, D) with constant D


def load_training_data(img_dir: Path, labels_map: dict):
    X, y, meta = [], [], []
    missing = 0
    for fname, pos_cells in labels_map.items():
        fpath = img_dir / fname
        if not fpath.exists():
            missing += 1
            continue
        img = cv2.imread(str(fpath), cv2.IMREAD_COLOR)
        if img is None:
            continue
        feats64 = extract_cell_features(img)  # (64, D)
        labels64 = np.zeros((GRID_W*GRID_H,), dtype=np.int32)
        for p in pos_cells:
            if 0 <= p < GRID_W*GRID_H:
                labels64[p] = 1
        X.append(feats64)
        y.append(labels64)
        meta.append(fname)
    if missing:
        print(f"[warn] {missing} labeled filenames not found in {img_dir}")
    X = np.vstack(X)         # (N*64, D)
    y = np.concatenate(y)    # (N*64,)
    return X, y, meta

def find_image_case_insensitive(img_dir: Path, fname: str) -> Path | None:
    """
    Returns a matching file path regardless of case (handles .jpg / .JPG / .jpeg differences).
    """
    fpath = img_dir / fname
    if fpath.exists():
        return fpath

    # Try case-insensitive match
    lower_name = fname.lower()
    for p in img_dir.iterdir():
        if p.is_file() and p.name.lower() == lower_name:
            return p
    return None

def build_per_image_tensors(img_dir: Path, labels_map: dict):
    per_img = []
    for fname, pos_cells in labels_map.items():
        fpath = find_image_case_insensitive(img_dir, fname) if 'find_image_case_insensitive' in globals() else (img_dir / fname)
        if not (fpath and Path(fpath).exists()):
            continue
        img = cv2.imread(str(fpath), cv2.IMREAD_COLOR)
        if img is None:
            continue
        feats64 = extract_cell_features(img)  # (64, D)
        labels64 = np.zeros((GRID_W*GRID_H,), dtype=np.int32)
        for p in pos_cells:
            if 0 <= p < GRID_W*GRID_H:
                labels64[p] = 1
        per_img.append((fname, feats64, labels64))
    return per_img


In [ ]:
# ========= REPLACE EVERYTHING BELOW WITH THIS BLOCK =========

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score, f1_score

# 1) Build per-image tensors (so we can split by image)
per_img = []
labels_map = read_labels(LABEL_CSV)

def _find_path(img_dir, fname):
    # use helper if you added it; else fall back to direct path
    if 'find_image_case_insensitive' in globals():
        p = find_image_case_insensitive(img_dir, fname)
        return p if p is not None else (img_dir / fname)
    return (img_dir / fname)

for fname, pos_cells in labels_map.items():
    fpath = _find_path(IMG_DIR, fname)
    if not fpath.exists():
        print(f"[warn] missing image for label: {fname}")
        continue
    img = cv2.imread(str(fpath), cv2.IMREAD_COLOR)
    if img is None:
        print(f"[warn] unreadable image: {fname}")
        continue
    feats64 = extract_cell_features(img)                # (64, D) — uses your (possibly padded) extractor
    labels64 = np.zeros((GRID_W*GRID_H,), dtype=np.int32)
    for p in pos_cells:
        if 0 <= p < GRID_W*GRID_H:
            labels64[p] = 1
    per_img.append((fname, feats64, labels64))

# 2) Split BY IMAGE
names = [x[0] for x in per_img]
train_names, test_names = train_test_split(names, test_size=0.20, random_state=RANDOM_SEED)

X_tr = np.vstack([x[1] for x in per_img if x[0] in train_names])
y_tr = np.concatenate([x[2] for x in per_img if x[0] in train_names])
X_te = np.vstack([x[1] for x in per_img if x[0] in test_names])
y_te = np.concatenate([x[2] for x in per_img if x[0] in test_names])

print("Train cells:", X_tr.shape[0], "| Test cells:", X_te.shape[0], "| Pos rate (train):", y_tr.mean().round(4))

# 3) Pipeline + Grid search over C
pipe_svm = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("svm", LinearSVC(class_weight="balanced", random_state=RANDOM_SEED, max_iter=8000))
])
param_grid = {"svm__C": [0.25, 0.5, 1.0, 2.0, 4.0]}
gs = GridSearchCV(pipe_svm, param_grid, scoring="f1", n_jobs=-1, cv=3, verbose=0)
gs.fit(X_tr, y_tr)
clf = gs.best_estimator_
print("Best params:", gs.best_params_)

# 4) Evaluate + tune decision threshold for best F1
scores_te = clf.decision_function(X_te)
# sweep thresholds between min and max decision score
thr_grid = np.linspace(scores_te.min(), scores_te.max(), 41)
best_f1, best_thr = 0.0, 0.0
for t in thr_grid:
    f1 = f1_score(y_te, (scores_te >= t).astype(int))
    if f1 > best_f1:
        best_f1, best_thr = f1, t

y_pred = (scores_te >= best_thr).astype(int)

print("\n=== Cell-level metrics (thresholded) ===")
print(classification_report(y_te, y_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_te, y_pred))
print(f"Chosen threshold: {best_thr:.4f}  (val F1={best_f1:.4f})")

# AUROC / AP (use raw scores)
print("AUROC:", roc_auc_score(y_te, scores_te))
print("Avg Precision (AP):", average_precision_score(y_te, scores_te))

# 5) Save model, config, and chosen threshold
joblib.dump(clf, MODEL_DIR / "hog_svm_grid.joblib")
with open(MODEL_DIR / "hog_svm_config.json", "w") as f:
    json.dump({
        "IMG_W": IMG_W, "IMG_H": IMG_H,
        "GRID_W": GRID_W, "GRID_H": GRID_H,
        "CELL_W": CELL_W, "CELL_H": CELL_H,
        "POS_INDICES_ARE_ONE_BASED": POS_INDICES_ARE_ONE_BASED,
        "HOG": {
            "orientations": HOG_ORIENTATIONS,
            "ppc": HOG_PIXELS_PER_CELL,
            "cpb": HOG_CELLS_PER_BLOCK,
            "norm": HOG_BLOCK_NORM
        },
        "USE_COLOR_HIST": USE_COLOR_HIST,
        "COLOR_BINS": [H_BINS, S_BINS, V_BINS]
    }, f, indent=2)

with open(MODEL_DIR / "best_threshold.txt", "w") as f:
    f.write(str(best_thr))

print("\nSaved model to:", MODEL_DIR / "hog_svm_grid.joblib")
print("Saved threshold to:", MODEL_DIR / "best_threshold.txt")
# ========= END REPLACEMENT =========



[warn] line 15: out-of-range indices [64] ignored for CIMG0006.JPG
[warn] line 48: out-of-range indices [64] ignored for CIMG0076.JPG
[warn] line 98: out-of-range indices [64] ignored for CIMG0118(2).JPG
[warn] line 101: out-of-range indices [64] ignored for CIMG0119.JPG
[warn] line 161: out-of-range indices [64] ignored for CIMG0163.JPG
[warn] line 180: out-of-range indices [64] ignored for CIMG0198.JPG
[warn] line 208: out-of-range indices [64] ignored for CIMG0242.JPG
[warn] line 242: out-of-range indices [64] ignored for CIMG0408.JPG
[warn] line 291: out-of-range indices [64] ignored for CIMG0602.JPG
[warn] line 330: out-of-range indices [64] ignored for IMG_20180724_170112733.jpg
[warn] line 350: out-of-range indices [64] ignored for IMG_20250709_163052484_HDR~2.jpg
[warn] line 351: out-of-range indices [64] ignored for IMG_20250709_164031681_HDR~2.jpg
[warn] line 378: out-of-range indices [64] ignored for IMG_3395.JPG
[warn] line 380: out-of-range indices [64] ignored for IMG_340